# neo4j graph fun

# Preliminaries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from pprint import pprint
import neo4j

In [3]:
import hashlib
from hashlib import sha256

In [4]:
from functools import lru_cache as cache

In [5]:
import ipywidgets as widgets
from nnvis import NetResponsePlot

In [6]:
from graph_utils_neo4j import NumpyStore
from nnbench import Thing

## Connecting

Need to get the `gpu-jupyter` and the `neo4j` docker containers connected. If run bare, something like:

    docker network connect gpu-jupyter_default gpu-jupyter 
    docker network connect gpu-jupyter_default neo4j
    docker network inspect gpu-jupyter_default 
    
Docker has better ways than this.

In [7]:
driver = neo4j.GraphDatabase.driver("neo4j://172.19.0.2:7687", auth=("neo4j", "test"))

### Alive?

In [8]:
driver.verify_connectivity()

<ipython-input-8-b734c674563a>:1: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


{IPv4Address(('172.19.0.2', 7687)): [{'ttl': 300,
   'servers': [{'addresses': ['172.19.0.2:7687'], 'role': 'WRITE'},
    {'addresses': ['172.19.0.2:7687'], 'role': 'READ'},
    {'addresses': ['172.19.0.2:7687'], 'role': 'ROUTE'}]}]}

# The Model
    Investigation -> Experiment -> multiple ResultDAGs
`ResultDAG` is

    (netState, params)-[mutation]->(netState, params)-[mutation ...
                     +-[mutation]->(netstate, params) ...
etc. `mutation` can be a learning trajectory, or an edit.

Perhaps `mutation` can be expressed in python.

Generally the results of experiments are preferred to be reproducible, but they won't always be, when they import entropy.

## Some neural nets

In [9]:
from nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from nnbench import NetMaker, NNMEG

In [10]:
mnm = NetMaker(NNMEG)
xor_net = mnm('2x2tx1t')
adc_net = mnm('1x8tx8tx3t')

## ... and training data

In [11]:
xor_training_batch = (np.array([[-0.5, -0.5],
                            [-0.5,  0.5],
                            [ 0.5,  0.5],
                            [ 0.5, -0.5]]),
                  np.array([[-0.5],
                            [ 0.5],
                            [-0.5],
                            [ 0.5]]))

In [12]:
def adc(input):
    m = max(0, min(7, int(8*input)))
    return np.array([(m>>2)&1, (m>>1)&1, m&1]) * 2 - 1

vadc = lambda v: np.array([adc(p) for p in v])
#plot_ADC(vadc)

In [13]:
x = np.arange(0, 1, 1.0/(8*1)).reshape(-1,1) # 1 point in each output region
adc_training_batch = (x, vadc(x))

In [16]:
def data_from_run():
    def get_sequence(tx, rv):
        for record in tx.run("MATCH p=(head:net)-[:LEARNED*]->(tail:net) "
                             "WHERE NOT ()-[:LEARNED]->(head) "
                             "AND NOT (tail)-[:LEARNED]->() "
                             "RETURN "
                             "head, "
                             "[x IN nodes(p) | x.ksv] as ksvs, "
                             "[x IN nodes(p) | x.loss] as losses "
                            ):
            rv.head = record['head']
            rv.ksvs = record['ksvs']
            rv.losses = record['losses']

    rv = Thing()
    with driver.session() as session:
        session.read_transaction(get_sequence, rv)
    return rv

In [17]:
run_data = data_from_run()

In [18]:
nps = NumpyStore(driver)

In [19]:
@cache
def sv_from_ksv(ksv):
    return nps[ksv]

In [20]:
net = mnm(run_data.head['shorthand'])

In [21]:
nrp = NetResponsePlot(net, height='220px', margin=30, title=net.shorthand)
nrp.fig

Figure(axes=[Axis(label='input', scale=LinearScale()), Axis(label='bits', orientation='vertical', scale=Linear…

In [23]:
#svl = [tget(rdl[0]['n.ksv'])] + [tget(rd['nn.ksv']) for rd in rdl]
#lossl = [rdl[0]['n.loss']] + [rd['nn.loss'] for rd in rdl]
def show_adc_n(n):
    nrp(sv_from_ksv(run_data.ksvs[n]))
    return run_data.losses[n]
#widgets.interact(show_adc_n, x=len(svl))
widgets.interact(show_adc_n, n=widgets.IntSlider(min=0, max=len(run_data.ksvs)-1, step=1, value=0));

interactive(children=(IntSlider(value=0, description='n', max=23), Output()), _dom_classes=('widget-interact',…

___

# Scrapyard

In [24]:
assert False, "Scrapyard below"

AssertionError: Scrapyard below

# Try Queries

In [14]:
def get_sequence(tx):
    for record in tx.run("MATCH p=(head:net)-[:LEARNED*]->(tail:net)"
                         "WHERE NOT ()-[:LEARNED]->(head)"
                         "AND NOT (tail)-[:LEARNED]->()"
                         "RETURN p, [x IN nodes(p) | x.ksv] as ksvs"):
        print(record['ksvs'])
        p = record['p']
        print(type(p))
        print(record['ksvs'])
        for r in p.relationships:
            print(r.start_node['ksv'], r['ts'], r.end_node['ksv'])

with driver.session() as session:
    session.read_transaction(get_sequence)

['1bdcf25bfffd2410', '966b55826105cd96', '8fa5dbeb0a2b71a1', 'd5495f6802b85342', '5e856706a1f52980', 'a527c6ed7e54d6c0', '357e79830534d7b3', 'af68e6458d007d13', 'a206ad679f12a646', '9d3fcbac469b3f46', 'da492f39da1afa6b', 'a4004dd380edcd91', '4717371f664840ce', 'c66a48c842e50787', 'dd388584a1d371c8', 'b4538cb909bf15be', 'dd81fa6c1a6e7bfd', '33cb3d9457a32665', '474e7d2f593623cf', 'b2e81e70d47d3d65', '0ff5746a24e68254', '5d195c1fd53cc8ee', '03404cdac4715bcd', '44a61817ca0c2a67']
<class 'neo4j.graph.Path'>
['1bdcf25bfffd2410', '966b55826105cd96', '8fa5dbeb0a2b71a1', 'd5495f6802b85342', '5e856706a1f52980', 'a527c6ed7e54d6c0', '357e79830534d7b3', 'af68e6458d007d13', 'a206ad679f12a646', '9d3fcbac469b3f46', 'da492f39da1afa6b', 'a4004dd380edcd91', '4717371f664840ce', 'c66a48c842e50787', 'dd388584a1d371c8', 'b4538cb909bf15be', 'dd81fa6c1a6e7bfd', '33cb3d9457a32665', '474e7d2f593623cf', 'b2e81e70d47d3d65', '0ff5746a24e68254', '5d195c1fd53cc8ee', '03404cdac4715bcd', '44a61817ca0c2a67']
1bdcf25bfff

In [15]:
def get_sequence(tx):
    for record in tx.run("MATCH p=(head:net)-[:LEARNED*]->(tail:net) "
                         "WHERE NOT ()-[:LEARNED]->(head) "
                         "AND NOT (tail)-[:LEARNED]->() "
                         "RETURN "
                         "head, "
                         "[x IN nodes(p) | x.ksv] as ksvs, "
                         "[x IN nodes(p) | x.loss] as losses "
                        ):
        head = record['head']
        ksvs = record['ksvs']
        losses = record['losses']
        print(ksvs, losses)
        print(head)
        
with driver.session() as session:
    session.read_transaction(get_sequence)

['1bdcf25bfffd2410', '966b55826105cd96', '8fa5dbeb0a2b71a1', 'd5495f6802b85342', '5e856706a1f52980', 'a527c6ed7e54d6c0', '357e79830534d7b3', 'af68e6458d007d13', 'a206ad679f12a646', '9d3fcbac469b3f46', 'da492f39da1afa6b', 'a4004dd380edcd91', '4717371f664840ce', 'c66a48c842e50787', 'dd388584a1d371c8', 'b4538cb909bf15be', 'dd81fa6c1a6e7bfd', '33cb3d9457a32665', '474e7d2f593623cf', 'b2e81e70d47d3d65', '0ff5746a24e68254', '5d195c1fd53cc8ee', '03404cdac4715bcd', '44a61817ca0c2a67'] [2.717673804175651, 1.8115520632018827, 1.1893921346954908, 0.85914990942073, 0.6100447583247612, 0.4334701657255896, 0.30867629626642257, 0.21927921769064634, 0.16059330568520738, 0.11160215842902874, 0.07664367125731167, 0.051935034821327275, 0.03487607937193718, 0.02552633593010603, 0.018048718874042517, 0.012761088088650719, 0.00902343938129117, 0.00638051464983168, 0.004511650800785589, 0.0031901056789924898, 0.002255666837562358, 0.0015949794115263998, 0.0011761980536661811, 0.0009171033080397184]
<Node id=2